# Autobus - Live demo

In [ ]:
#############import libraries, create helper functions, *.pth files, etc.#############
import time
print(time.ctime(),'go')

######Make Robot#######
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
robot = Robot()

######UI functions######
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display

######Monitor Network Connection#######
from jetbot import Heartbeat
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        print('check network connection')
heartbeat = Heartbeat(period=5)
heartbeat.observe(handle_heartbeat_status, names='status') # attach the callback function to heartbeat status

### Rail Tracking Setup

In [ ]:
######Rail Tracking Setup#######
import torchvision
import torch
steer_model_file = 'best_steering_model_xy--PoolTable400.pth'
steer_model = torchvision.models.resnet18(pretrained=False)
steer_model.fc = torch.nn.Linear(512, 2)
steer_model.load_state_dict(torch.load(steer_model_file))
steer_device = torch.device('cuda')
steer_model = steer_model.to(steer_device)
steer_model = steer_model.eval().half()
print(time.ctime(),steer_model_file,'ok')

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
steer_mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
steer_std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
def steer_preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(steer_device).half()
    image.sub_(steer_mean[:, None, None]).div_(steer_std[:, None, None])
    return image[None, ...]

#add the steering xy prediction to the image
def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

print(time.ctime(),'ok')

### Collision Avoidance Setup

In [ ]:
######Collision Avoidance Setup#######
import torch
import torchvision

oba_model = torchvision.models.alexnet(pretrained=False)
oba_model.classifier[6] = torch.nn.Linear(oba_model.classifier[6].in_features, 2)
#oba_model.load_state_dict(torch.load('best_model.pth'))
oba_model.load_state_dict(torch.load('best_obstacle_model--NVIDIA.pth'))
oba_device = torch.device('cuda')
oba_model = oba_model.to(oba_device)
print(time.ctime(),'oba_model','ok')

import cv2
import numpy as np

oba_mean = 255.0 * np.array([0.485, 0.456, 0.406])
oba_stdev = 255.0 * np.array([0.229, 0.224, 0.225])
oba_normalize = torchvision.transforms.Normalize(oba_mean, oba_stdev)
def oba_preprocess(camera_value):
    global oba_device, oba_normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = oba_normalize(x)
    x = x.to(oba_device)
    x = x[None, ...]
    return x

import torch.nn.functional as F
import time

print(time.ctime(),'ok')

### Create a function that will get called whenever the camera's value changes. 

In [ ]:
### Create a function that will get called whenever the camera's value changes. 
#This function will do the following steps
#- Pre-process the camera image
#- Execute the neural network
#- Stop and prevent motion on obstacle detection
#- Stop when jetbot reaches a station, resumes after a set time
#- If OK to move, control steering, keep jetbot on the track

######Global vars for execute#######
angle = 0.0
angle_last = 0.0
mt = 0
frame_cnt = 0
interval = 0.1
stutter = 1
debug = False
pp = time.time()

######Jetbot doing something with new image#######
def execute(change):
    global angle, angle_last, mt, frame_cnt, pp, debug, interval, accel, stutter, robot
    
    #count images read
    frame_cnt += 1
    
    #min time interval between each run
    if time.time() < pp + interval:
        return
    pp = time.time()
    
    #stutters robot, slows movement for turns
    if stutter:
        robot.left_motor.value = 0.1 *mt
        robot.right_motor.value = 0.1 *mt
        #time.sleep(0.1) #stop to get clear image
        
    #the new image
    image = change['new']

    ##AAAAAAAAAAA jetbot reads image and makes a obstacle stop decision ######
    x = change['new'] 
    x = oba_preprocess(x)
    y = oba_model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    if prob_blocked > 0.6:
        robot.left_motor.value = 0
        robot.right_motor.value = 0
        status_a.value = str(frame_cnt) + " obstacle prob: " + "{:.2f}".format(prob_blocked)
        return
    ##AAAAAAAAAAA jetbot finishes obstacle stop decision ######################
    

    
    
    ##BBBBBBBBBB jetbot reads image and makes a station stop decision #########
    
    # extracting out HSV color space values:
    # H – Hue (Dominant Wavelength)
    # S – Saturation (Purity/Shades of Color)
    # V – Value (Intensity)
    hsv = cv2.cvtColor(change['new'], cv2.COLOR_BGR2HSV)
    # adding median blur to image
    hsv = cv2.medianBlur(hsv,5)
    print(hsv)
    # lower range of red
    bttm_lower_red = np.array([0,120,70])
    top_lower_red = np.array([10,255,255])
    lower_red_mask = cv2.inRange(hsv, bttm_lower_red, top_lower_red)
    # upper range of red
    bttm_upper_red = np.array([170,120,70])
    top_upper_red = np.array([180,255,255])
    upper_red_mask = cv2.inRange(hsv, bttm_upper_red, top_upper_red)
    # generating the final mask to detect red color
    red_range_mask = lower_red_mask + upper_red_mask
    # removing noise through opening (erosion followed by dilation)
    red_range_mask = cv2.morphologyEx(red_range_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
    # increasing white region or size of foreground object increases through dilation
    red_range_mask = cv2.morphologyEx(red_range_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))
    # detecting if image frame contains red
    red_results = cv2.bitwise_and(image_frame, image_frame, mask = red_range_mask)
    # lower range of yellow
    bttm_lower_yellow = np.array([0,120,70]) # TODO: change values
    top_lower_yellow = np.array([10,255,255]) # TODO: change values
    lower_yellow_mask = cv2.inRange(hsv, bttm_lower_yellow, top_lower_yellow)
    # upper range of yellow
    bttm_upper_yellow = np.array([170,120,70]) # TODO: change values
    top_upper_yellow = np.array([180,255,255]) # TODO: change values
    upper_yellow_mask = cv2.inRange(hsv, bttm_upper_yellow, top_upper_yellow)
    # generating the final mask to detect yellow color
    yellow_range_mask = lower_yellow_mask + upper_yellow_mask
    # removing noise through opening (erosion followed by dilation)
    yellow_range_mask = cv2.morphologyEx(yellow_range_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
    # increasing white region or size of foreground object increases through dilation
    yellow_range_mask = cv2.morphologyEx(yellow_range_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))
    # detecting if image frame contains yellow
    yellow_results = cv2.bitwise_and(image_frame, image_frame, mask = yellow_range_mask)
    # lower range of green
    bttm_lower_green = np.array([0,120,70]) # TODO: change values
    top_lower_green = np.array([10,255,255]) # TODO: change values
    lower_green_mask = cv2.inRange(hsv, bttm_lower_green, top_lower_green)
    # upper range of green
    bttm_upper_green = np.array([170,120,70]) # TODO: change values
    top_upper_green = np.array([180,255,255]) # TODO: change values
    upper_green_mask = cv2.inRange(hsv, bttm_upper_green, top_upper_green)
    # generating the final mask to detect green color
    green_range_mask = lower_green_mask + upper_green_mask
    # removing noise through opening (erosion followed by dilation)
    green_range_mask = cv2.morphologyEx(green_range_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
    # increasing white region or size of foreground object increases through dilation
    green_range_mask = cv2.morphologyEx(green_range_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))
    # detecting if image frame contains green
    green_results = cv2.bitwise_and(image_frame, image_frame, mask = green_range_mask)
    # lower range of purple
    bttm_lower_purple = np.array([0,120,70]) # TODO: change values
    top_lower_purple = np.array([10,255,255]) # TODO: change values
    lower_purple_mask = cv2.inRange(hsv, bttm_lower_purple, top_lower_purple)
    # upper range of purple
    bttm_upper_purple = np.array([170,120,70]) # TODO: change values
    top_upper_purple = np.array([180,255,255]) # TODO: change values
    upper_purple_mask = cv2.inRange(hsv, bttm_purple_green, top_purple_green)
    # generating the final mask to detect purple color
    purple_range_mask = lower_purple_mask + upper_purple_mask
    # removing noise through opening (erosion followed by dilation)
    purple_range_mask = cv2.morphologyEx(purple_range_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
    # increasing white region or size of foreground object increases through dilation
    purple_range_mask = cv2.morphologyEx(purple_range_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))
    # detecting if image frame contains purple
    purple_results = cv2.bitwise_and(image_frame, image_frame, mask = purple_range_mask)
    if red_results is not None:
        # save detected color value as red
        color_detected = ‘red’
    elif yellow_results is not None:
        # save detected color value as yellow
        color_detected = ‘yellow’
    elif green_results is not None:
        # save detected color value as green
        color_detected = ‘green’
    elif purple_results is not None:
        # save detected color value as purple
        color_detected = ‘purple’
    else:
        # save detected color value as other
        color_detected = ‘other’
    # output deteced color value
    status_b.value = str(frame_cnt) + " color detected: " + color_detected
    
    ##BBBBBBBBBB jetbot finishes station stop decision #########

    
    ##CCCCCCCCCCC jetbot reads image and makes a steering decision ############
    xy = steer_model(steer_preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value

    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle

    steering_slider.value = pid + steering_bias_slider.value
   
    #use default NVIDA algo
    if not steer_debug:
        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0) *mt
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0) *mt
        interval = fps #set same as frame rate
        
        
    #########use AUTOBUS debugging algo##########
    else: 
        left_motor = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0) 
        right_motor = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0) 
        left_slider.value = left_motor
        right_slider.value = right_motor

        #run at preset speeds
        thres = turn_thres.value #steering value must exceed threshold to initate turning
        if steering_slider.value > thres:
            status_a.value = str(frame_cnt) + " turn right"
            robot.left_motor.value = max(robot.left_motor.value + 0.1, 0.4) *mt
            robot.right_motor.value = max(robot.right_motor.value + 0.1, 0.3) *mt
            interval = min(0.05, interval - 0.01) #decrease interval            
            stutter = 1
            
        elif steering_slider.value < -thres:
            status_a.value = str(frame_cnt) + " turn left"
            robot.left_motor.value = max(robot.left_motor.value + 0.1, 0.3) *mt
            robot.right_motor.value = max(robot.right_motor.value + 0.1, 0.45) *mt
            interval = min(0.05, interval - 0.01) #decrease interval            
            stutter = 1

        else: 
            status_a.value = str(frame_cnt) + " go straight"
            if robot.left_motor.value != robot.right_motor.value:
                robot.left_motor.value = robot.right_motor.value = min(robot.left_motor.value,robot.right_motor.value)
            #initial throttle then lower to coast speed    
            robot.left_motor.value = (robot.left_motor.value - 0.01) *mt
            robot.right_motor.value = (robot.right_motor.value - 0.01) *mt
            if min(robot.left_motor.value,robot.right_motor.value) < 0.2:
                robot.left_motor.value = robot.right_motor.value = 0.3 *mt  
            stutter = 0
            interval = max(interval+0.01, 0.1) #increase interval

        #set AUTOBUS adjusted value shown in UI
        left_adjusted.value = robot.left_motor.value
        right_adjusted.value = robot.right_motor.value
        ##CCCCCCCCCCCCCCCCCCC jetbot finished steering decision ##################
    
    return None
    
print(time.ctime(),'ok')

### setup UI - camera view, buttons, etc.

In [ ]:
#########setup interface - camera view, buttons, etc.#############

#user controlled sliders
#1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
#2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
#3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets
#jetbot turns when steering value exceeds threshold (for AUTOBUS steering algo only, not in NVIDIA steering algo)
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.4, step=0.01, value=0.33, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.4, step=0.01, value=0.33, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.4, step=0.01, value=0.33, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0, description='steering bias')
turn_thres = ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.25, description='threshold') #turn threshold

#buttons
button_layout = ipywidgets.Layout(width='80px', height='35px')
run_button = ipywidgets.Button(description='START', button_style='success', layout=button_layout)
run_button.on_click(lambda x: start())

stop_button = ipywidgets.Button(description='STOP', button_style='danger', layout=button_layout)
stop_button.on_click(lambda x: stop())

camera_button = ipywidgets.Button(description='CAMERA',layout=button_layout)
camera_button.on_click(lambda x: toggle_camera())

motor_button = ipywidgets.Button(description='MOTOR',layout=button_layout)
motor_button.on_click(lambda x: toggle_motor())

stutter_button = ipywidgets.Button(description='STUTTER',layout=button_layout)
stutter_button.on_click(lambda x: toggle_stutter())

#read only sliders
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, value=0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, value=0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

left_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='L nvidia')
right_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='R nvidia')

left_adjusted = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='L adjusted')
right_adjusted = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='R adjusted')

#The x and y sliders will display the predicted x, y values.
#The steering slider will display our estimated steering value.  Please remember, this value isn't the actual 
#angle of the target, but simply a value that is nearly proportional.  When the actual angle is ``0``, this 
#will be zero, and it will increase / decrease with the actual angle.  

#arrange widgets in UI
status_a = ipywidgets.Textarea(value = "status a")
status_b = ipywidgets.Textarea(value = "status b")
sliders = widgets.VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider,turn_thres])
buttons = widgets.VBox([run_button,stop_button,camera_button,motor_button,stutter_button])
y_sets = ipywidgets.HBox([y_slider,left_slider,right_slider,left_adjusted,right_adjusted])
x_sets = widgets.VBox([x_slider, steering_slider, status_a, status_b])

cam_toggle = 1 #enables/disables camera
def toggle_camera():
    global cam_toggle
    if cam_toggle:
        camera_link.unlink()
        cam_toggle = 0
    else:
        camera_link.link()
        cam_toggle = 1

mt = 0 #start/stop motor
def toggle_motor():
    global mt
    if mt: mt = 0
    else: mt = 1
        
stutter = 1 #enable/disable stutter-steps
def toggle_stutter():
    global stutter
    if stutter: stutter = 0
    else: stutter = 1

#start processing images
def start():
    execute({'new': camera.value})
    camera.observe(execute, names='value')

#stop jetbot, stop camera
def stop():
    time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
    camera.unobserve_all()
    robot.stop()
    camera.stop()

print(time.ctime(),'ok')

### Display UI

In [ ]:
try: camera.stop()
except: print('check camera')
fps = 10
camera = Camera.instance(width=224, height=224, capture_width=224, capture_height=224, fps=fps)
image_widget = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=display_xy)
camera.start()

frame_cnt = 0
mt = 0 #starts jetbot without moving
stutter = 0

steer_debug = False #False = default NVIDIA algo True = custom
turn_thres.value = 0.3 #only used if steer_debug=True
robot.left_motor.alpha = 1 #scales motor speed by this, 1 is no change
robot.right_motor.alpha = 1 

execute({'new': camera.value}) #test single image run
start() #start jetbot without moving (mt=0, MOTOR button to toggle movement)
print(time.ctime(),'ok')

display(widgets.HBox([image_widget,sliders,buttons]))
display(ipywidgets.HBox([y_sets,x_sets]))

#IMPORTANT: The camera must be re-initialized after it stops
#Rerun this block to reinitialize camera
#First run takes a few min to start up

///////////////////////////////////////////////End of Notebook //////////////////////////////////////////////////////////

### Debug

In [ ]:
#stop()

In [ ]:
#camera.stop()

In [ ]:
#robot.stop()

In [ ]:
#execute({'new': camera.value})
#camera.observe(execute, names='value')

In [ ]:
#start()